In [ ]:
#https://mkblog.co.kr/2019/01/19/python-sending-email-using-gamil-id/

In [3]:
import smtplib
from email.mime.text import MIMEText

In [4]:
def sendMail(me, you, msg):
    smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp.login(me, 'uyxhytdovynrzwcy')
    msg = MIMEText(msg)
    msg['Subject'] = 'TEST'
    smtp.sendmail(me, you, msg.as_string())
    smtp.quit()

In [5]:
sendMail('kusung25@gmail.com', 'kusung25@naver.com', '메일보내기')

In [6]:
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
# from email.message import Message
# from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.header import Header
import os

class MailManager:
    '''
    This class is used to send mail
    '''
    def __init__(self, emailfrom, emailto, subject, email_contents_path): #, logger=None        
        msg = MIMEMultipart()
        msg["From"]    = emailfrom
        msg['To']      = emailto
        msg['Subject'] = Header(subject,'utf-8') # 제목 인코딩
        msg.preamble   = "" #preamble #값을 넣지 말고 빈스트링으로 넘겨야 에러 안남
        #메일 본문 처리
        with open(email_contents_path, encoding="utf-8") as fp:
            # Create a text/html message
            content = MIMEText(fp.read(), 'html')
            #content = MIMEText(fp.read(), 'plain')            
            msg.attach(content)          
        self.msg = msg
        #self.logger = logger
    
    def attach_file(self, fileToSend):
        ctype, encoding = mimetypes.guess_type(fileToSend)
        if ctype is None or encoding is not None:
            ctype = "application/octet-stream"
        maintype, subtype = ctype.split("/", 1)

        if maintype == "text":
            fp = open(fileToSend)
            # Note: we should handle calculating the charset
            attachment = MIMEText(fp.read(), _subtype=subtype)
            fp.close()
        elif maintype == "image":
            fp = open(fileToSend, "rb")
            attachment = MIMEImage(fp.read(), _subtype=subtype)
            fp.close()
        else:
            fp = open(fileToSend, "rb")
            attachment = MIMEBase(maintype, subtype)
            attachment.set_payload(fp.read())
            fp.close()
            encoders.encode_base64(attachment)
        
        basename = os.path.basename(fileToSend)
        attachment.add_header("Content-Disposition", "attachment", filename=basename)
        self.msg.attach(attachment)
        return 'ok'
    def send_mail(self, fileToSend):
        self.attach_file(fileToSend)
        smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        me='kusung25@gmail.com'
        smtp.login(me, 'uyxhytdovynrzwcy')
        smtp.send_message(self.msg)
        smtp.quit()
        return 'ok'

In [7]:
emailfrom = "kusung25@gmail.com"
emailto   = "kusung25@naver.com" 
subject = "결과 송신 건"
email_contents_path = 'email_content.txt'
mm = MailManager(emailfrom, emailto, subject, email_contents_path)
ROOT_DIR= 'C:\\mylab\\book\\automate_basic' 
fileToSend = ROOT_DIR+'\\send.zip' #"send.zip"
mm.send_mail(fileToSend)

'ok'